<a href="https://colab.research.google.com/github/programmer-ayesha/Orchestration_Agent/blob/main/language_orchestration_agents_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔄 Multi-Agent Orchestration Workflow using OpenAI Agents SDK


In [2]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install -Uq "openai-agents[litellm]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.3 MB/s eta 0:00:00


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [6]:
Model = "gemini/gemini-2.0-flash"
api_key = gemini_api_key

In [8]:
# --- Setup and Imports ---
import asyncio
from agents import Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel

# Model & API key (Replace with your own)
Model = "gemini/gemini-2.0-flash"
api_key = gemini_api_key # 🔒 Keep secret

# --- Agent: Detect Language ---
detect_language_agent = Agent(
    name="Detect Language Agent",
    instructions="You are a language detection expert. Identify the input language of the user's message.",
    model=LitellmModel(model=Model, api_key=api_key),
)

# --- Agent: Spanish Translator ---
spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You translate the user's message to Spanish.",
    model=LitellmModel(model=Model, api_key=api_key),
)

# --- Agent: French Translator ---
french_agent = Agent(
    name="French Agent",
    instructions="You translate the user's message to French.",
    model=LitellmModel(model=Model, api_key=api_key),
)

# --- Agent: Orchestrator ---
# This agent decides what to do based on detected language
orchestrator_agent = Agent(
    name="Orchestrator Agent",
    instructions=(
        "You are a translation orchestrator. First, use the language detection tool "
        "to find the input language. Then call the appropriate translator tool to translate it to English. "
        "Use 'Spanish Agent' if it's Spanish, and 'French Agent' if it's French."
    ),
    model=LitellmModel(model=Model, api_key=api_key),
    tools=[
        detect_language_agent.as_tool(
            tool_name="detect_language",
            tool_description="Detect the language of the user's input",
        ),
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate message to French",
        ),
    ],
)

# --- Main Runner ---
async def main():
    input_text = "Say 'Hello, how are you?' in Spanish."  # 🔁 You can change input here
    result = await Runner.run(orchestrator_agent, input=input_text)
    print("✅ Final Output:", result.final_output)

# Run the async loop
asyncio.run(main())

✅ Final Output: Hola, ¿cómo estás?

